In [1]:
#from PIL import Image
import pytesseract
import cv2
import spacy
from spacy.matcher import Matcher
from rake_nltk import Rake
import os
import docx
import PyPDF2
import numpy as np 
import pandas as pd 


import sys
import nltk
import re
import requests
import subprocess
import pandas as pd
from tabulate import tabulate
#from pdfminer.high_level import extract_text
#from PyPDF2 import PdfReader, PdfFileWriter, PdfFileMerger


# Load Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Update with your Tesseract installation path

nlp = spacy.load('en_core_web_sm')

from nanonets import NANONETSOCR

In [13]:
#extract_text_from_pdf: This function takes a PDF or  file path as input and extracts text from all the pages in the PDF using NANONETSOCR

def extract_text_from_pdf_or_image(pdf_path):
    model = NANONETSOCR()
    model.set_token('87dcd5e7-53e9-11ee-bb5e-c26e11187d6d')
    text = model.convert_to_string(pdf_path,formatting='lines') 
    return text

# def extract_text_from_pdf(pdf_path):
#     with open(pdf_path, 'rb') as file:
#         pdf_reader = PyPDF2.PdfReader(file)
#         text = ''
#         for page in pdf_reader.pages:
#             text += page.extract_text()
#     return text

#extract_text_from_docx: This function takes a DOCX file path as input and extracts text from the document using the docx library.
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    text = '\n'.join([para.text for para in doc.paragraphs])
    return text

# #extract_text_from_image: This function takes an image file path as input, converts the image to grayscale, and then uses pytesseract to perform Optical Character Recognition (OCR) and extract text from the image.
# #Load the resume image using OpenCV and use OCR to extract text from the image
# def extract_text_from_image(image_path):
#     # Load the image using OpenCV
#     image = cv2.imread(image_path)
#     # Convert the image to grayscale
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     # Perform OCR to extract text
#     text = pytesseract.image_to_string(gray)
#     return text

In [9]:
def extract_contact_number_from_resume(text):
    contact_number = None

    # Use regex pattern to find a potential contact number
    pattern = r"\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"
    match = re.search(pattern, text)
    if match:
        contact_number = match.group()

    return contact_number

def extract_email_from_resume(text):
    email = None

    # Use regex pattern to find a potential email address
    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    match = re.search(pattern, text)
    if match:
        email = match.group()

    return email

def extract_skills_from_resume(text, skills_list):
    skills = []

    for skill in skills_list:
        pattern = r"\b{}\b".format(re.escape(skill))
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            skills.append(skill)

    return skills

def extract_education_from_resume(text):
    education = []

    # Use regex pattern to find education information
    pattern = r"(?i)(?:Bsc|\bB\.\w+|\bM\.\w+|\bPh\.D\.\w+|\bBachelor(?:'s)?|\bMaster(?:'s)?|\bPh\.D)\s(?:\w+\s)*\w+"
    matches = re.findall(pattern, text)
    for match in matches:
        education.append(match.strip())

    return education

def extract_name_from_resume(resume_text):
    nlp = spacy.load('en_core_web_sm')
    matcher = Matcher(nlp.vocab)

    # Define name patterns
    patterns = [
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name, Middle name, and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]  # First name, Middle name, Middle name, and Last name
        # Add more patterns as needed
    ]

    for pattern in patterns:
        matcher.add('NAME', patterns=[pattern])

    doc = nlp(resume_text)
    matches = matcher(doc)

    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text

    return None


In [23]:
def parse_resume(resume_text,file_name):
    skills_list = ['Python', 'Data Analysis', 'Machine Learning', 'Communication', 'Project Management', 'Deep Learning', 'SQL', 'Tableau']
    
    name = extract_name_from_resume(resume_text)
    email = extract_email_from_resume(resume_text)
    phone = extract_contact_number_from_resume(resume_text)
    skills = extract_skills_from_resume(resume_text,skills_list)
    education = extract_education_from_resume(resume_text)
    
 # Replace empty lists with a single empty string
    name = name or ['']
    email = email or ['']
    phone = phone or ['']
    skills = skills or ['']
    education = education or ['']
    info = {
        "File_name":file_name,
        "Name": name,
        "Email": email,
        "Phone": phone,
        "Skills": skills,
        "Education":education
    }
    return info



In [25]:
if __name__ == '__main__':
    # Directory containing candidate resumes (PDF, DOCX, and images)
    directory = 'C:/Users/user/Documents/GOMYCODE/ResumeParsing/dataset'
    AllresumeInfo=[]
    #resume_paths = [r"C:/Users/user/Documents/GOMYCODE/ResumeParsing/dataset/3.pdf"]
    #file_path =os.listdir(directory)
    #for filename in file_path:
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)        
        if os.path.isfile(file_path) and (file_path.lower().endswith('.pdf') or file_path.lower().endswith('.png') or file_path.lower().endswith('.jpg'))  :
            #resume_text = extract_text_from_pdf(file_path)
            resume_text=extract_text_from_pdf_or_image(file_path)
            info=parse_resume(resume_text,filename)
            AllresumeInfo.append(info)
        elif os.path.isfile(file_path) and file_path.lower().endswith('.docx'):
            resume_text = extract_text_from_docx(file_path)
            info=parse_resume(resume_text,filename)
            AllresumeInfo.append(info)    
        else:
            print('Unsupported File Format:', filename)
        continue
        

Unsupported File Format: 25.jfif
Unsupported File Format: 26.jfif
Unsupported File Format: 28.jpeg
Unsupported File Format: 29.jfif
Unsupported File Format: prog2.ipynb


In [26]:
# transformation en datafram
File_name=[]
Name =[]
Email =[]
Phone = []
Skills = []
Education = []
for i in range(len(AllresumeInfo)):
        File_name.append(AllresumeInfo[i]['File_name'])
        Name.append(AllresumeInfo[i]['Name'])
        Email.append(AllresumeInfo[i]['Email'])
        Phone.append(AllresumeInfo[i]['Phone'])
        Skills.append(AllresumeInfo[i]['Skills'])
        Education.append(AllresumeInfo[i]['Education'])
        
        
        
d = {
    'File_name':File_name, 
    'Name': Name, 
    'Email': Email, 
    'Phone':Phone,
    'Skills': Skills, 
    'Education':Education
    }
df = pd.DataFrame(data = d)
df.head()

,File_name,Name,Email,Phone,Skills,Education
0,1.pdf,Avishek Kumar,aviw.upes01@gmail.com,91999900011,"[Python, Data Analysis, Machine Learning, Deep...",[Ph.D Universiti\nAchievements\nTeknologi PETR...
1,10.pdf,Probabilistic Models,[],[],"[Python, Data Analysis, Machine Learning, Comm...",[]
2,11.pdf,Fulkar Khan,fulkarkhan9980@gmail.com,8443405050,"[Python, Machine Learning, Deep Learning, SQL,...",[]
3,12.pdf,Kunika Bhargav,bhargav496@gmail.com,7983924436,"[Python, Machine Learning, Deep Learning, SQL,...",[B.E in Electronics Engineering\nProiect focus...
4,13.pdf,Hugging Face,gkapil8019@gmail.com,9522330896,"[Python, Machine Learning, Deep Learning, SQL]",[Bachelor of Engineering]
